In [14]:
import os
import cv2
import numpy as np
import csv

# Function to save H4Pt to a CSV file
def save_h4pt(output_folder, output_filename, h4pt, index):
    h4pt_file = os.path.join(output_folder, "homography_gt.csv")
    # Append mode for writing the H4Pt values to the CSV file
    with open(h4pt_file, 'a', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow([f"{output_filename}_warped_{index}.jpg"] + list(h4pt.flatten()))

# Function to generate synthetic pairs of images with known homography and H4Pt labels
def generate_data(image_folder, output_folder, patch_size, max_perturbation, num_warped=2):
    # Create output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Initialize CSV file (overwrite if exists)
    h4pt_file = os.path.join(output_folder, "homography_gt.csv")
    with open(h4pt_file, 'w', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(['original_patch', 'warped_patch', 'x1', 'y1', 'x2', 'y2', 'x3', 'y3', 'x4', 'y4'])

    # Iterate through all images in the folder
    for filename in os.listdir(image_folder):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            # Read the image
            image_path = os.path.join(image_folder, filename)
            image = cv2.imread(image_path)
            if image is None:
                continue

            h, w = image.shape[:2]

            # Iterate to create multiple warped pairs from each image
            for i in range(num_warped):
                # Random patch size within the specified range
                patch_height = np.random.randint(patch_size[0]//2, patch_size[0])
                patch_width = np.random.randint(patch_size[1]//2, patch_size[1])

                # Randomly select the top-left corner of the patch
                x = np.random.randint(0, w - patch_width)
                y = np.random.randint(0, h - patch_height)
                patch = image[y:y + patch_height, x:x + patch_width]
                patch_corners = np.array([[x, y], [x + patch_width, y], [x, y + patch_height], [x + patch_width, y + patch_height]])

                # Generate random perturbation within [-max_perturbation, max_perturbation]
                perturbation = np.random.uniform(-max_perturbation, max_perturbation, size=(4, 2))
                perturbed_corners = patch_corners + perturbation

                # Calculate homography between original and perturbed corners
                h_matrix, _ = cv2.findHomography(patch_corners, perturbed_corners)

                # Calculate H4Pt labels
                h4pt = perturbed_corners - patch_corners

                # Warp the original image using the inverse of the homography
                warped_image = cv2.warpPerspective(image, np.linalg.inv(h_matrix), (w, h))

                # Extract corresponding patch from the warped image
                warped_patch = warped_image[y:y + patch_height, x:x + patch_width]

                # Save the original and warped patches as separate images
                output_filename = os.path.splitext(filename)[0]
                cv2.imwrite(os.path.join(output_folder, f"{output_filename}_original_{i}.jpg"), patch)
                cv2.imwrite(os.path.join(output_folder, f"{output_filename}_warped_{i}.jpg"), warped_patch)

                # Save the H4Pt values to the CSV file
                save_h4pt(output_folder, output_filename, h4pt, i)

# Example usage
image_folder = "../Data/Val"
output_folder = "../Data/Val_generated_data"
patch_size = (100, 100)  # Maximum size of the patch to extract
max_perturbation = 20  # Maximum perturbation range [-max_perturbation, max_perturbation]

generate_data(image_folder, output_folder, patch_size, max_perturbation, num_warped=5)

image_folder = "../Data/Train"
output_folder = "../Data/Train_generated_data"
generate_data(image_folder, output_folder, patch_size, max_perturbation, num_warped=5)


KeyboardInterrupt: 

In [1]:
import os
import cv2
import numpy as np
import csv

def save_h4pt(output_folder, output_filename, h4pt, index):
    h4pt_file = os.path.join(output_folder, "homography_gt.csv")
    with open(h4pt_file, 'a', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow([f"{output_filename}_warped_{index}.jpg"] + list(h4pt.flatten()))

def generate_data(image, patch_size=128, max_perturbation=24):
    h, w = image.shape[:2]
    x = np.random.randint(patch_size // 2, w - patch_size // 2)
    y = np.random.randint(patch_size // 2, h - patch_size // 2)
    
    original_patch = image[y - patch_size // 2:y + patch_size // 2, x - patch_size // 2:x + patch_size // 2]
    
    corners = np.array([[x - patch_size // 2, y - patch_size // 2],
                        [x + patch_size // 2, y - patch_size // 2],
                        [x - patch_size // 2, y + patch_size // 2],
                        [x + patch_size // 2, y + patch_size // 2]], dtype=np.float32)
    
    perturbation = np.random.uniform(-max_perturbation, max_perturbation, size=(4, 2)).astype(np.float32)
    perturbed_corners = corners + perturbation

    homography_ab = cv2.getPerspectiveTransform(corners, perturbed_corners)
    warped_image = cv2.warpPerspective(image, np.linalg.inv(homography_ab), (w, h))

    warped_patch = warped_image[y - patch_size // 2:y + patch_size // 2, x - patch_size // 2:x + patch_size // 2]
    
    return original_patch, warped_patch, perturbation, corners, homography_ab

def generate_DataSet(image_folder, output_folder, patch_size=128, max_perturbation=24, num_warped=10):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    orig_output_dir = os.path.join(output_folder, 'Original')
    warp_output_dir = os.path.join(output_folder, 'Warped')
    labels_output_dir = output_folder

    if not os.path.exists(orig_output_dir):
        os.makedirs(orig_output_dir)
    if not os.path.exists(warp_output_dir):
        os.makedirs(warp_output_dir)
    
    perturbation_list = []
    homography_list = []
    corners_list = []
    count = 0

    for filename in os.listdir(image_folder):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            image_path = os.path.join(image_folder, filename)
            image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            if image is None:
                continue
            
            for i in range(num_warped):
                original_patch, warped_patch, perturbation, corners, homography_ab = generate_data(image, patch_size, max_perturbation)
                
                cv2.imwrite(os.path.join(orig_output_dir, f"{count}_{i}_original.jpg"), original_patch)
                cv2.imwrite(os.path.join(warp_output_dir, f"{count}_{i}_warped.jpg"), warped_patch)
                
                perturbation_list.append(perturbation)
                homography_list.append(homography_ab)
                corners_list.append(corners)
                
                count += 1

    np.save(os.path.join(labels_output_dir, 'Perturbations.npy'), perturbation_list)
    np.save(os.path.join(labels_output_dir, 'Homographies.npy'), homography_list)
    np.save(os.path.join(labels_output_dir, 'Corners.npy'), corners_list)


image_folder = "../Data/Val"
output_folder = "../Data/Val_generated_data"
generate_DataSet(image_folder, output_folder, patch_size=128, max_perturbation=24, num_warped=5)

# image_folder = "../Data/Train"
# output_folder = "../Data/Train_generated_data"
# generate_DataSet(image_folder, output_folder, patch_size=128, max_perturbation=24, num_warped=5)
